<a href="https://colab.research.google.com/github/RichardHarwood/3D_Unet_Plant_Roots/blob/main/3D_Unet_from_source.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wget
!pip install open3d
!pip install vedo

In [2]:
import os
!git clone https://github.com/wolny/pytorch-3dunet.git

fatal: destination path 'pytorch-3dunet' already exists and is not an empty directory.


In [3]:
  %cd pytorch-3dunet

/content/pytorch-3dunet


In [4]:
!python setup.py install

running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https

In [5]:
import torch
torch.cuda.is_available()

True

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
#need to set appropriate  home folder
home_folder = "/content/drive/MyDrive/3D_UNET/"

In [8]:
#Create Config Folder
import os.path
from os import path
if path.exists(home_folder+'config_files') == False:
  os.mkdir(home_folder+'config_files')
config_folder = home_folder+'config_files/'

In [22]:
#Create Checkpoint Folder
import os.path
from os import path
if path.exists(home_folder+'checkpoint_dir') == False:
  os.mkdir(home_folder+'checkpoint_dir')
checkpoint_dir = home_folder+'checkpoint_dir/'

In [10]:
#https://zenodo.org/records/4766931

In [11]:
#!pip install wget
# import wget
# wget.download("https://zenodo.org/records/4766931/files/"+"E camaldulensis - Cropped.tif"+"?download=1", out='/content/drive/MyDrive/3D_UNET/download_data/euc.tif')

In [12]:
#Create folder to store zenodo data

Choose all the settings here

In [23]:
model_name = "UNet3D"
in_channels = 1
out_channels= 1
layer_order = "gcr"
f_maps = [32,
          64,
          128,
          256]
num_groups = 8
final_sigmoid = True
is_segmentation = True
resume = None
pre_trained = None
validate_after_iters = 500
log_after_iters = 500
max_num_epochs = 10  ##Low for testing
max_num_iterations = 1800000
eval_score_higher_is_better = True
loss_name= 'BCEWithLogitsLoss'
learning_rate= 0.0002
weight_decay= 0.00001
eval_name= "MeanIoU"
lr_name= "ReduceLROnPlateau"
mode= 'max'
factor = 0.2
patience = 30
dataset= "StandardHDF5Dataset"
batch_size= 3  #important for gpu memory
num_workers= 2
raw_internal_path= "raw"
label_internal_path= "label"
weight_internal_path= None
################3
file_paths_train= "/content/drive/MyDrive/3D_UNET/training_visit_3_transfer/train/"
######################
slice_builder_name= "SliceBuilder"
slice_builder_name_predict = "SliceBuilder"
patch_shape= [40, 170, 170]  #Change depending on GPU memory
stride_shape= [20, 40, 40]   #Change depending on GPU memory
halo_shape= [16, 32, 32]
threshold = 0.01
slack_acceptance = 0.01
name_transformer= "Standardize"
name_transformer_label= "BlobsToMask"
append_label= False
boundary= False
ToTensor_name = "ToTensor"
expand_dims_false= False
expand_dims_true= True
################
file_paths_val= "/content/drive/MyDrive/3D_UNET/training_visit_3_transfer/qc/"
#####################
predictor_name='StandardPredictor'
#################################
file_paths_test= "/content/drive/MyDrive/3D_UNET/training_visit_3_transfer/test/"
#############################
model_path=checkpoint_dir+"best_checkpoint.pytorch"

Code below creates .yaml file

In [24]:
# @title
#https://stackoverflow.com/questions/12470665/how-can-i-write-data-in-yaml-format-in-a-file
#https://stackoverflow.com/questions/59034564/yaml-dump-seeming-to-add-two-dashes-to-2nd-row-under-first-key


import yaml
train_intrem_config = {'model':{'name':model_name,
                                'in_channels':in_channels,
                                'out_channels':out_channels,
                                'layer_order':layer_order,
                                'f_maps':f_maps,
                                'num_groups':num_groups,
                                'final_sigmoid':final_sigmoid,
                                'is_segmentation':is_segmentation},
                       'trainer': {'checkpoint_dir':checkpoint_dir,
                                   'resume':resume,
                                   'pre_trained':pre_trained,
                                   'validate_after_iters':validate_after_iters,
                                   'log_after_iters':log_after_iters,
                                   'max_num_epochs':max_num_epochs,
                                   'max_num_iterations':max_num_iterations,
                                   'eval_score_higher_is_better':eval_score_higher_is_better},
                       'loss':{'name':loss_name},
                       'optimizer':{'learning_rate':learning_rate,
                                    'weight_decay':weight_decay},
                       'eval_metric':{'name':eval_name},
                       'lr_scheduler':{'name':lr_name,
                                       'mode':mode,
                                       'factor':factor,
                                       'patience':patience},
                       'loaders':{'dataset':dataset,
                                       'batch_size':batch_size,
                                       'num_workers':num_workers,
                                       'raw_internal_path':raw_internal_path,
                                       'label_internal_path':label_internal_path,
                                       'weight_internal_path':weight_internal_path,
                       'train':{'file_paths':[file_paths_train],"slice_builder":{'name':slice_builder_name,
                                                                               'patch_shape':patch_shape,
                                                                               'stride_shape':stride_shape,
                                                                               'threshold':threshold,
                                                                               'slack_acceptance':slack_acceptance},
                       'transformer': {"raw": [{"name":name_transformer},
                                               {"name":ToTensor_name,
                                                "expand_dims":expand_dims_true}],
                                       "label": [{"name":name_transformer_label,
                                                "append_label":append_label},
                                               {"name":ToTensor_name,
                                                "expand_dims":expand_dims_false}]}},
                       'val':{"file_paths":[file_paths_val],"slice_builder":{'name':slice_builder_name,
                                                                               'patch_shape':patch_shape,
                                                                               'stride_shape':patch_shape,
                                                                               'threshold':threshold,
                                                                               'slack_acceptance':slack_acceptance},
                       'transformer': {"raw": [{"name":name_transformer},
                                               {"name":ToTensor_name,
                                                "expand_dims":expand_dims_true}],
                                       "label": [{"name":name_transformer_label,
                                                  "append_label":append_label},
                                               {"name":ToTensor_name,
                                                "expand_dims":expand_dims_false}]}}}}

with open(config_folder+'model_train.yaml', 'w') as yaml_file:
    yaml.dump(train_intrem_config, yaml_file, default_flow_style=False, sort_keys=False)

 #print(yaml.dump(train_intrem_config, default_flow_style=False, sort_keys=False))

Copy the text above and paste it after --config e.g. !predict3dunet --config /content/drive/MyDrive/3D_UNET/config_files/model_train.yaml

if you want to use a previous model do not run the below cell or better yet # it out. If using an already trained model change

model_path=checkpoint_dir+"best_checkpoint.pytorch" accordingly

In [ ]:
!train3dunet --config /content/drive/MyDrive/3D_UNET/config_files/model_train.yaml

2024-10-15 00:58:29.314687: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-15 00:58:29.336023: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-15 00:58:29.342365: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-15 00:58:29.357398: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-15 00:58:30.554958: W tensorflow/comp

In [ ]:
test_config_yaml ={
    "model_path": model_path,
    "model": {
        "name": model_name,
        "in_channels": in_channels,
        "out_channels": out_channels,
        "layer_order": layer_order,
        "f_maps": f_maps,
        "num_groups": num_groups,
        "final_sigmoid": final_sigmoid,
        "is_segmentation": is_segmentation
    },
    "predictor": {
        "name": predictor_name
    },
    "loaders": {
        "batch_size": batch_size,
        "raw_internal_path": raw_internal_path,
        "num_workers": num_workers,
        "test": {
            "file_paths": [
                file_paths_test
            ],
            "slice_builder": {
                "name": slice_builder_name_predict,
                "patch_shape": patch_shape,
                "stride_shape": patch_shape,
                "halo_shape": halo_shape
            },
            "transformer": {
                "raw": [
                    {
                        "name": name_transformer
                    },
                    {
                        "name": ToTensor_name,
                        "expand_dims": expand_dims_true
                    }
                ]
            }
        }
    }
}

with open(config_folder+'predict.yaml', 'w') as yaml_file:
    yaml.dump(test_config_yaml, yaml_file, default_flow_style=False, sort_keys=False)


In [ ]:
print(yaml.dump(test_config_yaml, default_flow_style=False, sort_keys=False))

In [ ]:
!predict3dunet --config /content/drive/MyDrive/3D_UNET/config_files/predict.yaml

In [ ]:
import glob
import numpy as np
raw_data_path= file_paths_test
raw_file_names = [os.path.basename(x) for x in glob.glob(raw_data_path+"*.h5")]
raw_file_names = [e for e in raw_file_names if "predictions" in e ]
#################################
file_paths_processed= "/content/drive/MyDrive/3D_UNET/training_visit_3_transfer/processed/"
#########################
raw_file_names

In [ ]:
UNIQUE_ID =np.unique(raw_file_names)
len(UNIQUE_ID)

In [ ]:

import h5py
import cv2
import skimage
from skimage.filters import threshold_otsu, threshold_yen
from skimage.measure import label
from skimage.morphology import remove_small_objects
from vedo import *
import pyvista as pv
for i in UNIQUE_ID:
    print(i)
    hf=h5py.File(raw_data_path+i)
    hf.keys()
    dataset_IMGs= hf['predictions']
    image=dataset_IMGs[:]
    image=image[0,:,:,:]
    #image=zoom(image, (scale_factor,scale_factor, scale_factor), order=1)
    image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    thresh = threshold_yen(image)-45
    #thresh=20
    print (thresh)
    binary = image > thresh
    #binary=zoom(binary, (scale_factor,scale_factor, scale_factor), order=0)
    root_sml=label(binary)
    min_size_coef=2500 #trial and error really
    root_sml_obj = remove_small_objects(root_sml, min_size=min_size_coef, connectivity=2)
    root_sml_obj = (root_sml_obj != 0).astype(bool)

    root_VOL = Volume(root_sml_obj, spacing=((1),(1),(1)))
    root_ISO = root_VOL.isosurface(1)
    root_ISO.write(file_paths_processed+i+"__root.stl")
    skimage.io.imsave(file_paths_processed+i+"_processed.tif", root_sml_obj, check_contrast=False)

Now, to sanity check lets create a 3D model of the root system we just segmented using 3D-Unet

In [ ]:
import open3d as o3d
import plotly.graph_objects as go
model_to_test="recon_sub__103_wheat_pvc_4days_R3_wb_1x_stitch.tif_014_predictions.h5__root.stl"
mesh = o3d.io.read_triangle_mesh(file_paths_processed+model_to_test)
if not mesh.has_vertex_normals(): mesh.compute_vertex_normals()
if not mesh.has_triangle_normals(): mesh.compute_triangle_normals()
triangles = np.asarray(mesh.triangles)
vertices = np.asarray(mesh.vertices)
colors = None

colors = (1.0, 0.0, 0.0)

fig = go.Figure(
    data=[
        go.Mesh3d(
            x=vertices[:,0],
            y=vertices[:,1],
            z=vertices[:,2],
            i=triangles[:,0],
            j=triangles[:,1],
            k=triangles[:,2],
            facecolor=colors,
            opacity=0.50)
    ],
    layout=dict(
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False)
        )
    )
)
fig.show()